In [49]:
from game import TicTacToe
import random
import math
import time
infinity = math.inf

# 1. Getting Started with the Game

In [50]:
# initializing a TicTacToe game
game = TicTacToe(height=3, width=3, k=3)
board = game.initial
# get the possible actions
actions = game.actions(board)
# to apply a move (the board automatically figures out which player)
board_after_move = game.result(board, list(actions)[0])

print(board)
print(actions)
print(board_after_move)

. . .
. . .
. . .

{(0, 1), (1, 2), (2, 1), (0, 0), (1, 1), (2, 0), (0, 2), (2, 2), (1, 0)}
. . .
X . .
. . .



In [51]:
def play_game_step(game, state, strategies: dict, verbose=False):
    start = time.perf_counter()
    player = state.to_move
    move = strategies[player](game, state)
    state = game.result(state, move)
    time_elapsed = time.perf_counter() - start
    if verbose: 
        print('Player', player, 'move:', move, f'time: {time_elapsed:.4f}s', )
        print(state)
    return state

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.is_terminal(state):
        state = play_game_step(game, state, strategies, verbose)
    return state

# setup a random strategy for testing
def random_player(game, state): return random.choice(list(game.actions(state)))

def search_player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1] # we expect our search algorithm to return (v, move)

In [52]:
# initialize a game
game = TicTacToe()
state = game.initial
strategies = dict(X=random_player, O=random_player)

In [53]:
# option 1: 
# play_game(game, strategies, True)

# option 2: run it step by step by executing this cell multiple times until the game end
if not game.is_terminal(state):
    state = play_game_step(game, state, strategies, True)
else:
    print(f"Game result: {game.utility(state, 'X')}")
    print(state)

Player X move: (0, 0) time: 0.0001s
X . .
. . .
. . .



# 2. MINIMAX and Alpha-Beta 

### MINIMAX-DECISION and EXPECTIMINIMAX

__function__ MINIMAX-DECISION(_state_) __returns__ _an action_  
&emsp;__return__ arg max<sub> _a_ &Element; ACTIONS(_s_)</sub> MIN\-VALUE(RESULT(_state_, _a_))  

---
__function__ MAX\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__function__ EXPECTIMINIMAX(_s_) =     
&emsp;UTILITY(_s_) __if__ TERMINAL\-TEST(_s_)  
&emsp;max<sub>_a_</sub> EXPECTIMINIMAX(RESULT(_s, a_)) __if__ PLAYER(_s_)= MAX  
&emsp;min<sub>_a_</sub> EXPECTIMINIMAX(RESULT(_s, a_)) __if__ PLAYER(_s_)= MIN  
&emsp;∑<sub>_r_</sub> P(_r_) EXPECTIMINIMAX(RESULT(_s, r_)) __if__ PLAYER(_s_)= CHANCE  


In [54]:
def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""

    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v < v2:
                v = v2
                move = a
            # TODO: decide *v* and *move*
            
        return v, move

    def min_value(state):
        # TODO: implement function min_value
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v > v2:
                v = v2
                move = a
        return v, move

    return max_value(state)

# test against random_player
%time play_game(TicTacToe(), dict(X=search_player(minimax_search), O=random_player), True)

Player X move: (0, 1) time: 5.6661s
. . .
X . .
. . .

Player O move: (1, 0) time: 0.0000s
. O .
X . .
. . .

Player X move: (0, 0) time: 0.0743s
X O .
X . .
. . .

Player O move: (2, 0) time: 0.0000s
X O O
X . .
. . .

Player X move: (1, 2) time: 0.0013s
X O O
X . .
. X .

Player O move: (2, 2) time: 0.0000s
X O O
X . .
. X O

Player X move: (2, 1) time: 0.0001s
X O O
X . X
. X O

Player O move: (0, 2) time: 0.0000s
X O O
X . X
O X O

Player X move: (1, 1) time: 0.0001s
X O O
X X X
O X O

CPU times: total: 5.61 s
Wall time: 5.75 s


X O O
X X X
O X O

In [55]:
# if you have implemented it right, it will always be a draw for h=w=k=3
%time play_game(TicTacToe(), dict(X=search_player(minimax_search), O=search_player(minimax_search)), True)

Player X move: (0, 1) time: 5.9903s
. . .
X . .
. . .

Player O move: (2, 1) time: 0.7495s
. . .
X . O
. . .

Player X move: (1, 2) time: 0.1062s
. . .
X . O
. X .

Player O move: (0, 0) time: 0.0164s
O . .
X . O
. X .

Player X move: (1, 1) time: 0.0025s
O . .
X X O
. X .

Player O move: (1, 0) time: 0.0004s
O O .
X X O
. X .

Player X move: (2, 0) time: 0.0001s
O O X
X X O
. X .

Player O move: (0, 2) time: 0.0001s
O O X
X X O
O X .

Player X move: (2, 2) time: 0.0000s
O O X
X X O
O X X

CPU times: total: 6.7 s
Wall time: 6.87 s


O O X
X X O
O X X

### ALPHA-BETA-SEARCH

__function__ ALPHA-BETA-SEARCH(_state_) __returns__ an action  
&emsp;_v_ &larr; MAX\-VALUE(_state_, &minus;&infin;, &plus;&infin;)  
&emsp;__return__ the _action_ in ACTIONS(_state_) with value _v_  

---
__function__ MAX\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &ge; _&beta;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&alpha;_ &larr; MAX(_&alpha;_, _v_)  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &plus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &le; _&alpha;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&beta;_ &larr; MIN(_&beta;_, _v_)  
&emsp;__return__ _v_  


In [56]:
def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v < v2:
                v = v2
                move = a

            if v>=beta:
                return v, move
            alpha = max(v, alpha)
            # TODO: update alpha, beta pruning, decide *v* and *move*
            
        return v, move

    def min_value(state, alpha, beta):
        # TODO: implement function min_value
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v > v2:
                v = v2
                move = a

            if v <= alpha:
                return v, move
            beta = min(v, beta)
            
        return v, move

    return max_value(state, -infinity, +infinity)

# test against random_player
%time play_game(TicTacToe(), dict(X=search_player(alphabeta_search), O=random_player), True)

Player X move: (0, 1) time: 0.2509s
. . .
X . .
. . .

Player O move: (2, 2) time: 0.0000s
. . .
X . .
. . O

Player X move: (0, 2) time: 0.0166s
. . .
X . .
X . O

Player O move: (0, 0) time: 0.0000s
O . .
X . .
X . O

Player X move: (1, 1) time: 0.0007s
O . .
X X .
X . O

Player O move: (2, 0) time: 0.0000s
O . O
X X .
X . O

Player X move: (2, 1) time: 0.0002s
O . O
X X X
X . O

CPU times: total: 281 ms
Wall time: 268 ms


O . O
X X X
X . O

In [57]:
%time play_game(TicTacToe(), dict(X=search_player(alphabeta_search), O=search_player(alphabeta_search)), True)

Player X move: (0, 1) time: 0.2517s
. . .
X . .
. . .

Player O move: (2, 1) time: 0.0534s
. . .
X . O
. . .

Player X move: (1, 2) time: 0.0144s
. . .
X . O
. X .

Player O move: (0, 0) time: 0.0038s
O . .
X . O
. X .

Player X move: (1, 1) time: 0.0014s
O . .
X X O
. X .

Player O move: (1, 0) time: 0.0003s
O O .
X X O
. X .

Player X move: (2, 0) time: 0.0003s
O O X
X X O
. X .

Player O move: (0, 2) time: 0.0002s
O O X
X X O
O X .

Player X move: (2, 2) time: 0.0002s
O O X
X X O
O X X

CPU times: total: 312 ms
Wall time: 327 ms


O O X
X X O
O X X

# 3. Heuristic Cutoffs and Monte-Carlo Tree Search(Optional)

In [58]:
def cutoff_depth(d):
    """A cutoff function that searches to depth d."""
    return lambda game, state, depth: depth > d

def h_alphabeta_search(cutoff=cutoff_depth(6), h=lambda s, p: 0):
    def search(game, state):
        """
        Search game to determine best action; use alpha-beta pruning.
        As in [Figure 5.7], this version searches all the way to the leaves.
        TODO: add checks for *cutoff* and return *h* at proper locations
        """

        player = state.to_move

        def max_value(state, alpha, beta, depth):
            if game.is_terminal(state):
                return game.utility(state, player), None

            v, move = -infinity, None
            for a in game.actions(state):
                v2, _ = min_value(game.result(state, a), alpha, beta, depth+1)
                if v < v2:
                    v = v2
                    move = a

                if v>=beta:
                    return v, move
                alpha = max(v, alpha)
                # TODO: update alpha, beta pruning, decide *v* and *move*
                
            return v, move

        def min_value(state, alpha, beta, depth):
            # TODO: implement function min_value
            if game.is_terminal(state):
                return game.utility(state, player), None
            v, move = +infinity, None
            for a in game.actions(state):
                v2, _ = max_value(game.result(state, a), alpha, beta, depth+1)
                if v > v2:
                    v = v2
                    move = a

                if v <= alpha:
                    return v, move
                beta = min(v, beta)
            
            return v, move

        return max_value(state, -infinity, +infinity, 0)
    return search

# plug in your own *h* and see its performance against a random player
game = TicTacToe()

# I'll just cheat with the true utility for a demo (won't work when the tree is deeper)
h = lambda s, p: game.utility(s, p) 
cutoff = cutoff_depth(9)

%time play_game(game, {'X':search_player(h_alphabeta_search(cutoff=cutoff, h=h)), 'O':random_player}, True)

Player X move: (0, 1) time: 0.2615s
. . .
X . .
. . .

Player O move: (0, 2) time: 0.0000s
. . .
X . .
O . .

Player X move: (1, 2) time: 0.0133s
. . .
X . .
O X .

Player O move: (2, 0) time: 0.0000s
. . O
X . .
O X .

Player X move: (1, 1) time: 0.0007s
. . O
X X .
O X .

Player O move: (1, 0) time: 0.0000s
. O O
X X .
O X .

Player X move: (2, 1) time: 0.0001s
. O O
X X X
O X .

CPU times: total: 281 ms
Wall time: 276 ms


. O O
X X X
O X .

### MONTE-CARLO-TREE-SEARCH(Optional)

__function__ MONTE-CARLO-TREE-SEARCH(_state_) __returns__ an action  
&emsp;tree &larr; NODE(_state_)
&emsp;__while__ TIME\-REMAINING() __do__  
&emsp;&emsp;&emsp;__tree__ &larr; PLAYOUT(_tree_)  
&emsp;__return__ the _move_ in ACTIONS(_state_) with highest Q(_state_,_move_)  

---

__function__ PLAYOUT(_tree_) __returns__ _updated tree_  
&emsp;_node_ &larr; _tree_  
&emsp;__while__ _node_ is not terminal and was already in _tree_ __do__  
&emsp;&emsp;&emsp;_move_ &larr; SELECT(_node_)  
&emsp;&emsp;&emsp;_node_ &larr; FOLLOW\-LINK(_node_,_move_)  
&emsp;_outcome_ &larr; SIMULATION(_node_.STATE)  
&emsp;UPDATE(_node_,_outcome_)  
&emsp;__return__ _tree_  

---

__function__ SELECT(_node_) __returns__ _an action_  
&emsp;__return__ argmax<sub>m &isin; FEASIBLE\-ACTIONS(_node_)</sub> UCB(RESULT(_node_,_m_))  

---

__function__ UCB(_child_) __returns__ _a number_  
&emsp;__return__ _child_.VALUE + C &times; <a href="https://www.codecogs.com/eqnedit.php?latex=\inline&space;\sqrt{\frac{\log{child.PARENT.N}}{child.N}}" target="_blank"><img src="https://latex.codecogs.com/png.latex?\inline&space;\sqrt{\frac{\log{child.PARENT.N}}{child.N}}" title="\sqrt{\frac{\log{child.PARENT.N}}{child.N}}" /></a> 

---

The Monte Carlo tree search algorithm. A game tree, _tree_, is initialized, and then grows by one node with each call to PLAYOUT. The function SELECT chooses a move that best balances exploitation and exploration according to the UCB formula. FOLLOW-LINK traverses from the current node by making a move; this could be to a previously-seen node, or to a new node that is added to the tree. Once we have added a new node, we exit the __while__ loop and SIMULATION chooses moves (with a randomized policy that is designed to favor good moves but to compute quickly) until the game is over. Then, UPDATE updates all the nodes in the tree from node to the root, recording the fact that the path led to the final __outcome__.

In [59]:
import numpy as np
class MCT_Node:
    """Node in the Monte Carlo search tree, keeps track of the children states."""

    def __init__(self, parent=None, state=None, U=0, N=0):
        self.__dict__.update(parent=parent, state=state, U=U, N=N)
        self.children = {} # dict of {node: action} or {actione: node}
        self.actions = None

def ucb(n, C=1.4):
    return np.inf if n.N == 0 else n.U / n.N + C * np.sqrt(np.log(n.parent.N) / n.N)

def monte_carlo_tree_search(game, state, N=1000):
    def select(n):
        """
        select a leaf node in the tree according to their UCB values
        """
        if n.children:
            return select(max(n.children.keys(), key=ucb))
        else:
            return n

    def expand(n):
        """ 
        expand the leaf node by adding all its children states (FOLLOW-LINK)
        """
        if not n.children and not game.is_terminal(n.state):
            n.children = {None: None for action in game.actions(n.state)} # TODO: fix this
        return select(n)

    def simulate(game, state):
        """ 
        simulate the utility of current state using a random strategy
        """
        player = state.to_move
        while not game.is_terminal(state):
            # TODO: fix and complete this
            state = state
            
        v = 0
        return v

    def backprop(n, utility):
        """
        passing the utility back to all parent nodes
        """
        if utility > 0:
            n.U += utility
        n.N += 1
        if n.parent:
            backprop(n.parent, -utility)

    root = MCT_Node(state=state)

    for _ in range(N):
        # PLAYOUT
        leaf = select(root)
        child = expand(leaf)
        result = simulate(game, child.state)
        backprop(child, result)
    
    # TODO: select the action
    v, move = 0, (0, 0)
    return v, move

# if you have done it right it should usually win
%time play_game(game, {'X':search_player(monte_carlo_tree_search), 'O':random_player}, True)

AttributeError: 'NoneType' object has no attribute 'N'

# 4. Further Hints on Project

* Be **absolutely sure** that you are not maximizing your opponent's utility!
* Code optimization: the performance of a python program greatly depends on its implementation. Write your code to be as efficient as possible and maybe search for performance hacks!
* Performance tuning: gather statistics to help yourself in deciding the hyperparameters, e.g.:
  * +
  * ...